https://github.com/SeleniumHQ/docker-selenium

docker run -d -p 4444:4444 -p 7900:7900 --shm-size="2g" selenium/standalone-firefox:latest

Point your WebDriver tests to http://localhost:4444
To see what is happening inside the container, head to http://localhost:7900 (password is secret)


In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.by import By

In [2]:
options = FirefoxOptions()
driver = webdriver.Remote(
    command_executor='http://172.17.0.2:4444',
    options=options
)

In [134]:
driver.get('https://www.mikroe.com/click')
productMenu=driver.find_element(By.XPATH, "(//*[contains(@class, 'tree dynamized')])")
clickBoardsMenu=productMenu.find_element(By.PARTIAL_LINK_TEXT,"Click Boards").find_element(By.XPATH, "..")

In [135]:
x=clickBoardsMenu.find_element(By.XPATH, "//ul")

In [136]:
y = x.find_elements(By.TAG_NAME,"li")

In [145]:
y[5].find_element(By.TAG_NAME,"a").get_attribute("href")

'https://www.mikroe.com/news'

In [34]:
y=clickBoardsMenu.find_elements(By.XPATH,'//a[contains(@href,"https://www.mikroe.com/click/")]')

In [117]:
z1=driver.find_element(By.CLASS_NAME,"tree dynamized")
#z2=z1.find_element(By.PARTIAL_LINK_TEXT,"Click Boards")
#z3=z2.find_element(By.XPATH,"./..")
#z4=z1.find_element(By.XPATH,"//li[1]/ul")
#z4=z3.find_elements(By.XPATH, "(//*[contains(@class, 'grower')])")
#z5=z4.find_elements(By.TAG_NAME, "li")

NoSuchElementException: Message: Unable to locate element: .tree dynamized
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:183:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:395:5
element.find/</<@chrome://remote/content/marionette/element.js:300:16


In [101]:
z3.tag_name

'li'

In [112]:
len(z5)

759

In [103]:
driver.get('https://www.mikroe.com/click/sensors/rf-meter')
products=driver.find_elements(By.XPATH, "(//*[contains(@class, 'product-container')])")
for product in products:
    code=product.find_element(By.CSS_SELECTOR, "div.content_price span.product-reference").get_attribute("textContent")
    name=product.find_element(By.CLASS_NAME, "product-name").get_attribute("textContent")
    url=product.find_element(By.CLASS_NAME, "product-name").get_attribute("href")
    description=product.find_element(By.CLASS_NAME, "product-desc").get_attribute("textContent")
    price=product.find_element(By.CLASS_NAME, "me-product-price").get_attribute("textContent") 

In [100]:
driver.quit()

In [ ]:
# things = driver.find_elements(By.XPATH, "//div[@class='af_pl_wrapper']")
# things = driver.find_elements(By.CSS_SELECTOR, "li.ajax_block_product")
# things = driver.find_elements(By.CSS_SELECTOR, "div.product-container.clearfix")
# things = driver.find_elements(By.XPATH, "(//*[contains(@class, 'product-container')])")
# x = things[0].find_element(By.CSS_SELECTOR, "p.product-desc")